In [89]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [10]:
SEQ_LEN = 30
INTERVAL = 5
assert 0 < INTERVAL < 60 

In [11]:
TRAIN = 0.7
VAL = 0.15
TEST = 0.15
assert TRAIN + VAL + TEST == 1

#### Load data

In [122]:
df_og = pd.read_csv('../sample_data/avg_from_2020_10_01_00_00_00_to_2020_10_01_23_59_00')
df_og['czas'] = pd.to_datetime(df_og['czas'])
df_og.set_index('czas', inplace=True)

temp_zuz = pd.read_csv('../sample_data/temp_zuz.csv', delimiter=';')
temp_zuz['Czas'] = pd.to_datetime(temp_zuz['Czas'])

In [123]:
df_og.head()

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s
czas,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00+02:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.090454,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87
2020-10-01 00:01:00+02:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.090534,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87
2020-10-01 00:02:00+02:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.090613,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87
2020-10-01 00:03:00+02:00,54.154394,54.985713,12.052065,22.335388,108.623473,112.386061,109.836948,108.717994,110.464245,104.418573,...,92.090692,92.024462,92.405153,24.661329,24.526161,4.48,8.98,8.6,4.55,9.87
2020-10-01 00:04:00+02:00,54.693184,54.490742,12.027310,22.412620,108.746784,112.396792,109.966910,108.978605,110.603660,104.464328,...,92.090771,92.024590,92.405110,24.660003,24.525475,4.48,8.98,8.6,4.55,9.87


### Proste featury wybieram albo tworze agregując


In [109]:
df = pd.DataFrame(index=df_og.index)

In [110]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = df_og.columns[df_og.columns.str.contains('tir')].values
df['TIR'] = df_og[TIR].mean(axis='columns')

In [111]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = df_og.columns[df_og.columns.str.contains('001tix')].values
df['TIX1'] = df_og[TIX1].mean(axis='columns')

In [112]:
# wrzucam o prażonkach i prob
PR = df_og.columns[df_og.columns.str.contains('pr')].values
df[PR] = df_og[PR]

### Features to pct change

In [114]:
df = df.pct_change()

In [115]:
df

,TIR,TIX1,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s
czas,,,,,,,
2020-10-01 00:00:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-01 00:01:00+02:00,0.000055,-0.000002,0.0,0.0,0.0,0.0,0.0
2020-10-01 00:02:00+02:00,0.000235,-0.000002,0.0,0.0,0.0,0.0,0.0
2020-10-01 00:03:00+02:00,0.000141,-0.000002,0.0,0.0,0.0,0.0,0.0
2020-10-01 00:04:00+02:00,-0.000073,-0.000002,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2020-10-01 23:55:00+02:00,0.000065,-0.000003,0.0,0.0,0.0,0.0,0.0
2020-10-01 23:56:00+02:00,0.000065,-0.000003,0.0,0.0,0.0,0.0,0.0
2020-10-01 23:57:00+02:00,0.000091,-0.000003,0.0,0.0,0.0,0.0,0.0


### Dołaczam temp_zuz

In [124]:
df.merge(temp_zuz, left_index=True

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [ ]:
df_train

### Scale

In [90]:
scaler = StandardScaler().fit(df.values)

scaler 

scaler.mean_, scaler.scale_

(array([ 1.40832063e-05,  2.92765034e-07, -8.19884739e-07,  2.53059455e-05,
         4.85375100e-05,  1.24999025e-05, -1.67298044e-05]),
 array([2.80688131e-04, 3.33724557e-06, 2.13742725e-03, 6.73598836e-03,
        2.22946179e-03, 8.83775053e-04, 8.66521320e-04]))